### Create a Translator Text Service
To see this in action, you need to provision a Text Analytics service in your Azure subscription. Follow these steps to do that:

1. Open another browser tab and navigate to https://portal.azure.com.
2. Sign in using your Microsoft account.
3. Click **+ New**, and in the **AI + Cognitive Services** category, click **See all**.
4. In the list of cognitive services, click **Translator Text API**.
5. In the **Translator Text API** blade, click **Create**.
6. In the **Create** blade, enter the following details, and then click **Create**
  * **Name**: A unique name for your service.
  * **Subscription**: Your Azure subscription.
  * **Pricing tier**: Choose the F0 pricing tier.
  * **Resource Group**: Choose an existing resource group or create a new one.
  * Read the notice about the use of your data, and select the checkbox.
7. Wait for the service to be created.
8. When deployment is complete, click **All Resources** and then click your Translator Text service to open its blade.
9. In the blade for your service, click **Keys** and then copy **Key 1** to the clipboard and paste it into the **translatorKey** variable assignment value in the cell below. 
11. Run the cell below to assign the variable.

In [ ]:
translatorKey = '51ef61e196ab420db9cf50004bcdde04'

Now we can initialize a TranslatorService object that will handle all API calls to the service

In [ ]:
from translator import TranslatorService
translator = TranslatorService(translatorKey)

## translate

Below you can see examples of translating from 3 languages into english

Notice the simple usage: **translator.translate(someText, language)** 

In [ ]:
# use the translate function
textToTranslate = 'Esta frase deverá ser correctamente traduzida para inglês'
print('\nTranslating from portuguese: \"%s\"' % textToTranslate)
result = translator.translate(textToTranslate, 'en')
print ('Result: %s' % result)

textToTranslate = 'Este servicio de traducción es genial!'
print('\nTranslating from spanish: \"%s\"' % textToTranslate)
result = translator.translate(textToTranslate, 'en')
print ('Result: %s' % result)

textToTranslate = 'Het kan zelfs vertalen Nederlandse'
print('\nTranslating from dutch: \"%s\"' % textToTranslate)
result = translator.translate(textToTranslate, 'en')
print ('Result: %s' % result)

The second parameter of the function we created is the intended language to translate to. The Microsoft Translator API also allows to pass in a parameter specifying the original language but the service is usually able to detect it automatically so this implementation does not include it.

In [ ]:
textToTranslate = 'It can translate to other languages too'
print('\nTranslating from english: \"%s\"' % textToTranslate)
result = translator.translate(textToTranslate, 'es')
print ('Result: %s' % result)

There are actually two models we can use for translation.

The legacy model is called **Statistical Machine Translation (SMT)** and can be further customized using the Translator Hub (https://hub.microsofttranslator.com/SignIn?returnURL=%2FHome%2FIndex).

The new model is called **Neural Machine Translation (NMT)** and used Deep Neural Networks for the translation. We have to specify it explicitly on the API calls, but the wrapper does that automatically. In this case, if you want to use SMT you need to specify **use_nn=False** on the translate method call.

Here's an example:

In [ ]:
#Using NMT vs SMT (nerual machine translation vs Statistical machine translation)
textToTranslate = 'Esta é uma frase particularmente complexa escrita em português e deverá produzir resultados diferentes conforme a tecnologia seleccionada para a tradução'
print('Translating from portuguese: \"%s\"\n' % textToTranslate)

result = translator.translate(textToTranslate, 'en')
print ('Result with NMT: \n%s' % result)

result2 = translator.translate(textToTranslate, 'en', use_nn=False)
print ('Result with SMT: \n%s\n' % result2)

In the results above we can see NMT was slightly better: **'should produce'** is more accurate than '**must produce** in this context as the original phrase in portuguese implies that it is likely we'll get different results as opposed to an absolute certainty.

## translateArray

The TranslatorService class also implements the translateArray which allows to send a list of strings to translate in a single call. Usage is similar to the translate method, except it now requires a list of strings as the first parameter:

**translator.translateArray(listOfStrings, language)**

In [ ]:
li = []
li.append('Vamos tentar traduzir várias frases de uma vez só')
li.append('A ordem é mantida na resposta mas a linguagem de todas as frases tem de ser a mesma')
li.append('Esta tradução está a fazer uso de redes neuronais para melhorar os resultados')

print('Using NMT')
result = translator.translateArray(li,'en')
for s in result:
    print(s)

print('\nUsing SMT')
result = translator.translateArray(li,'en', use_nn=False)
for s in result:
    print(s)

## detect

The service is also able to detect in which language a particular piece of text is written. The wrapper implements the **detect** function which can be used for this:

usage: **translator.detect(someText)**

In [ ]:
textToDetect = "Será que a detecção funciona?"
result = translator.detect(textToDetect)
print('Detected language: {0}'.format(result))

## detectArray

As for translations, we can also send multiple strings for language detection

usage: **translator.detectArray(listOfStrings)**

In [ ]:
li = []
li.append("Let's try detecting the language of several strings")
li.append("Esta está em português")
li.append("et celui-ci en français")

translator.detectArray(li)

## Speak

You can instruct the api to return a wave or mp3 stream of the passed-in text being spoken in the desired language

In [ ]:
text = 'Python is fun!'
translator.speak(text,"en-us","female")

fr_text = translator.translate(text, "fr-fr")
translator.speak(fr_text,"fr-fr","female")

## getLanguagesForTranslate | getLanguageForSpeak | getLanguageNames


Finally, the wrapper also includes methods to get all the supported language codes (which you can use on the previous methods) and also the friendly name associated to those codes for displaying in a UI. Usage: 

**translator.getLanguagesForTranslate()**

**translator.getLanguagesForSpeak()**

**translator.getLanguageNames(listOfLangCodes, locale)**

In [ ]:
langCodes = translator.getLanguagesForTranslate()
print("Language codes (translation):")
print(langCodes)

langCodes = translator.getLanguagesForSpeak()
print("\nLanguage codes (speak):")
print(langCodes)

languageNames = translator.getLanguageNames(langCodes,'en')
print("\nLanguage names:")
print(languageNames)


### Just for fun

A small interactive application when you run this cell, so you can freely test the service

In [ ]:
import random
import numpy as np

textToTranslate = input("Input a string in any supported language:")

langCodes = translator.getLanguagesForSpeak()
langFriendlyNames = translator.getLanguageNames(langCodes,'en')

#choose 5 random languages
random_values = (list(np.arange(len(langCodes))))
sample = random.sample(random_values, 5)

print ("\nLanguage List\n")
i = 0
for l in sample:
    i = i + 1
    print('%i:%s' % (i, langFriendlyNames[l]))

toLangCode = " "
# Get the destination language
while (toLangCode == " "):
    destLang = int(input("\nChoose the language from the list that you want to translate to: "))
    if (destLang >= 1) or (destLang <= 5):
        toLangCode = langCodes[sample[destLang-1]]
    else:
        print ("\nYou need to pick a language from the List\n")
 
result = translator.translate(textToTranslate, toLangCode)
print('\nTranslation: %s' % result)
translator.speak(textToTranslate, toLangCode)
